In [83]:
# imports
import requests
from bs4 import BeautifulSoup
import json
from collections import namedtuple
from tqdm import tqdm
import pandas as pd

# constants
ICMC = namedtuple("ICMC", ["statusTitle", "id", "category", "subCategory", "title", "pictures", "priority", "cityId", "landmark", "name", "longitude", "latitude", "civicAgency", "description", "wardId", "slug", "url", "postedOn", "wardTitle"])
output_csv = "../icmc_output.csv"
limit = 10000

In [84]:
# base url

url = "https://api.ichangemycity.com/api/complaints/list?agency=0&category_id=0&city=1&include=comments&latitude=0&longitude=0&page={}&status_id=0&sub_category=0&type=list"

In [87]:
# encoding the dataframe to utf-8

def encode_dataframe(df):
    df_columns = df.columns.tolist()
    for c in df_columns: 
        if df[c].dtype != 'int64':
            df[c] = df[c].str.encode('utf-8')
    return df

In [ ]:
icmc = []

for idx, i in tqdm(enumerate(range(limit))):
    try:
        data = json.loads(requests.get(url.format(i + 1)).text)["data"]
        for d in data:
            icmc.append(ICMC(d["statusTitle"], d["id"], d["category"], d["subCategory"], d["title"], d["pictures"], d["priority"], d["location"]["cityId"], d["location"]["landmark"], d["location"]["name"], d["location"]["longitude"], d["location"]["latitude"], d["civicAgency"], d["description"], d["wardId"], d["slug"], d["url"], d["postedOn"], d["wardTitle"]))
    except Exception as e:
         print("Exception occurred: {}".format(e.message))
    
    if (idx + 1) % 10 == 0:
        df_ = pd.DataFrame(icmc)
        with open(output_csv, 'a') as fp:
            encode_dataframe(df_)
            df_.to_csv(fp, header=False, index=False)
        icmc = []
        
df_ = pd.DataFrame(icmc)
with open(output_csv, 'a') as fp:
    encode_dataframe(df_)
    df_.to_csv(fp, header=False, index=False)